In [ ]:
%pip install -U azure-search-documents pandas

In [2]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI, OpenAI

NEXTGEN_API_TYPE: bool = True

def print_responses_output(responses: object) -> None:
    print("=== Responses Output ===")
    print(f"🧾 ID: {responses.id}")
    print(f"🧾 Model: {responses.model}")
    print(f"🧾 Status: {responses.status}")
    print(f"🧾 Usage: {responses.usage}")
    for output in responses.output:
        if output.type == "reasoning":
            print(f"🤔 Reasoning: {output.summary}")
        elif output.type == "message":
            for content in output.content:
                if content.type == "output_text":
                    print(f"💬 Message|OutputText: {content.text}")
    print("========================")

# notebook 환경변수를 위해 .env 파일을 로드합니다.
load_dotenv(dotenv_path=".env", override=True)

# OpenAI 의 Azure 클라이언트를 생성합니다.
if NEXTGEN_API_TYPE:
    client = OpenAI(
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        base_url=os.getenv("AZURE_OPENAI_ENDPOINT") + "openai/v1/",
        default_query={"api-version": "preview"},
    )
else:
    client = AzureOpenAI(
        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    )

# GPT-5 OpenAI 의 Azure 클라이언트를 생성합니다.
if NEXTGEN_API_TYPE:
    gpt5_client = OpenAI(
        api_key=os.getenv("AZURE_OPENAI_GPT5_API_KEY"),
        base_url=os.getenv("AZURE_OPENAI_GPT5_ENDPOINT") + "openai/v1/",
        default_query={"api-version": "preview"},
    )
else:
    gpt5_client = AzureOpenAI(
        api_version=os.getenv("AZURE_OPENAI_GPT5_API_VERSION"),
        azure_endpoint=os.getenv("AZURE_OPENAI_GPT5_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_GPT5_API_KEY"),
    )

In [3]:
# input/output message content를 캡처하기 위해 환경변수를 설정합니다.
import os

os.environ["OTEL_INSTRUMENTATION_GENAI_CAPTURE_MESSAGE_CONTENT"] = "true"
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"

# from azure.ai.projects import AIProjectClient
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor
from azure.ai.projects import AIProjectClient
from azure.identity import AzureCliCredential

# OpenAI API 호출을 자동으로 계측하기 위해 OpenAIInstrumentor를 사용합니다.
OpenAIInstrumentor().instrument()

# AIProjectClient를 사용하여 Azure AI Foundry 프로젝트에 연결합니다.
# API Key를 TokenCredential로 변환합니다.
project_client = AIProjectClient(
    credential=AzureCliCredential(),
    endpoint=os.getenv("AZURE_AI_FOUNDRY_PROJECT_ENDPOINT"),
)
# Azure Monitor를 구성합니다.
configure_azure_monitor(connection_string=project_client.telemetry.get_application_insights_connection_string())

# RAG Pipeline using AI Search

## Establish Connection

In [4]:
from typing import Final
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    HnswAlgorithmConfiguration,
    SearchableField,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    VectorSearch,
    VectorSearchProfile,
)

INDEX_NAME: Final[str] = "receipts"

# Azure Search Index 클라이언트를 생성합니다.
search_index_client = SearchIndexClient(
    endpoint=os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
    credential=AzureKeyCredential(os.getenv("AZURE_AI_SEARCH_API_KEY")),
)
# 인덱스가 존재하는지 확인하고, 있으면 삭제하고 새로 생성합니다.
found = INDEX_NAME in [index.name for index in search_index_client.list_indexes()]
if found:
    search_index_client.delete_index(INDEX_NAME)

search_index_client.create_index(
    SearchIndex(
        name=INDEX_NAME,
        fields=[
            SimpleField(name="rid", type=SearchFieldDataType.String, key=True),
            SearchableField(name="merchantName", type=SearchFieldDataType.String),
            SearchableField(name="transactionDate", type=SearchFieldDataType.Int64), # String 강제
            SimpleField(name="transactionTime", type=SearchFieldDataType.Int64, filterable=True, sortable=True),
            SimpleField(name="tax", type=SearchFieldDataType.Double, filterable=True, sortable=True),
            SimpleField(name="total", type=SearchFieldDataType.Double, filterable=True, sortable=True),
            SearchField(
                name="descEmbedding",
                type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                vector_search_dimensions=3072,
                vector_search_profile_name="desc-embedding-profile",
            ),
        ],
        vector_search=VectorSearch(
            profiles=[
                VectorSearchProfile(
                    name="desc-embedding-profile",
                    algorithm_configuration_name="desc-embedding-algorithm-config",
                )
            ],
            algorithms=[
                HnswAlgorithmConfiguration(name="desc-embedding-algorithm-config"),
            ]
        )
    )
)

# Azure AI Search 클라이언트를 생성합니다.
search_client = SearchClient(
    endpoint=os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
    index_name=INDEX_NAME,
    credential=AzureKeyCredential(os.getenv("AZURE_AI_SEARCH_API_KEY")),
)

## Data Preprocessing

In [5]:
import pandas as pd
from uuid import uuid4

# receipts.csv 파일을 읽어옵니다. 이 파일은 receipts 데이터셋을 포함합니다.
receipts = pd.read_csv("resources/receipts/receipts.csv")

# sum all expenses
print(f"Total expenses: {receipts['total'].sum()} won")

# embedding 을 수행할 chunks 를 정의합니다.
chunks = [
    (
        f'Receipt from {receipt["merchantName"]} on {receipt["transactionDate"]} '
        f'at {receipt["transactionTime"]}. ' if receipt["transactionTime"] else '. '
        f'Total price is {receipt["total"]} with tax {receipt["tax"]}.'
    ) for _, receipt in receipts.iterrows()
]

receipts.head()

Total expenses: 15852300 won


,merchantName,transactionDate,transactionTime,transactionYear,transactionMonth,transactionWeek,transactionDay,transactionDayOfWeek,tax,total
0,코스트코,20251008,192919,2025,10,41,8,3,19120,191200
1,교보문고,20250821,163930,2025,8,34,21,4,16470,164700
2,롯데백화점,20250206,71917,2025,2,6,6,4,19920,199200
3,스타벅스,20250606,205133,2025,6,23,6,5,26430,264300
4,명랑핫도그,20250918,114519,2025,9,38,18,4,9190,91900


In [6]:
from openai import AzureOpenAI

# receipts.iloc 에 embedding 생성합니다.
response = client.embeddings.create(
    model=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
    input=chunks,
)
# documents 로 변환합니다.
documents = [
    {
        "rid": uuid4().hex,
        "merchantName": receipts.iloc[data.index]["merchantName"],
        "transactionDate": str(receipts.iloc[data.index]["transactionDate"]),
        "transactionTime": int(receipts.iloc[data.index]["transactionTime"]),
        "tax": float(receipts.iloc[data.index]["tax"]),
        "total": float(receipts.iloc[data.index]["total"]),
        "descEmbedding": data.embedding,
    } for data in response.data
]
# documents 를 업로드합니다.
search_client.upload_documents(documents)

In [9]:
from azure.search.documents.models import VectorizedQuery

response = client.embeddings.create(
    model=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
    input="20250821",
)

# vector search 실행
results = search_client.search(
    search_text=None,
    vector_queries=[
        VectorizedQuery(
            vector=response.data[0].embedding,
            k_nearest_neighbors=3,
            fields="descEmbedding",
        ),
    ],
)
print("=== Search Results using vector search ===")
for result in results:
    print(f"Merchant: {result['merchantName']}, Date: {result['transactionDate']}, "
          f"Time: {result['transactionTime']}, Tax: {result['tax']}, Total: {result['total']}")
    
# vector search 실행
results = search_client.search(
    search_text="20250821",
    search_fields=["transactionDate"],
)
print("=== Search Results using keyword search ===")
for result in results:
    print(f"Merchant: {result['merchantName']}, Date: {result['transactionDate']}, "
          f"Time: {result['transactionTime']}, Tax: {result['tax']}, Total: {result['total']}")

=== Search Results using vector search ===
Merchant: 김밥천국, Date: 20250821, Time: 173229, Tax: 19830.0, Total: 198300.0
Merchant: 교보문고, Date: 20250821, Time: 163930, Tax: 16470.0, Total: 164700.0
Merchant: GS25, Date: 20250205, Time: 55828, Tax: 19970.0, Total: 199700.0
=== Search Results using keyword search ===
Merchant: 김밥천국, Date: 20250821, Time: 173229, Tax: 19830.0, Total: 198300.0
Merchant: 교보문고, Date: 20250821, Time: 163930, Tax: 16470.0, Total: 164700.0


# AI Agent with AI Search

In [10]:
from textwrap import dedent
from typing import Annotated
from azure.search.documents.models import VectorizedQuery
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions.kernel_function_decorator import kernel_function


class ExpensesPlugin:
    search_client = SearchClient(
        endpoint=os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
        index_name=INDEX_NAME,
        credential=AzureKeyCredential(os.getenv("AZURE_AI_SEARCH_API_KEY")),
    )
    azure_open_ai = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    )
    
    @kernel_function(name="get_total_expenses", description="Get total expenses.")
    async def get_total_expenses(self) -> float:
        results = self.search_client.search(
            search_text=None,
            select="total",
        )
        total_expenses = sum(result["total"] for result in results)
        return total_expenses

    @kernel_function(name="get_expenses", description="Get expenses for a given data.")
    async def get_expenses_by_date(
        self,
        data: Annotated[str, "Data to get expenses as date or merchant name. Use '20250821' for date or 'CU' for merchant name."],
    ) -> list[dict]:
        return self.vector_search(data)

    def vector_search(self, query: str) -> list[dict]:
        # This function performs a vector search on the Azure AI Search index.
        response = self.azure_open_ai.embeddings.create(
            model=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
            input=query,
        )
        results = self.search_client.search(
            search_text=None,
            vector_queries=[
                VectorizedQuery(
                    vector=response.data[0].embedding,
                    k_nearest_neighbors=3,
                    fields="descEmbedding",
                ),
            ],
        )
        return [result for result in results]
    
    
class BudgetAmountPlugin:
    user_budget: dict[str, float] = {}

    @kernel_function(name="set_budget_amount", description="Set a budget amount for the user.")
    async def set_user_budget_amount(
        self,
        user: Annotated[str, "The name of the user"],
        amount: Annotated[float, "The budget amount to set"],
    ) -> None:
        # This function sets the budget amount for the user.
        self.user_budget[user] = amount
        
    @kernel_function(name="get_budget_amount", description="Get the budget amount for the user.")
    async def get_user_budget_amount(
        self,
        user: Annotated[str, "The name of the user"],
    ) -> float:
        # This function retrieves the budget amount for the user.
        if user in self.user_budget:
            return self.user_budget[user]
        else:
            print(f"No budget amount set for {user}.")
            return 0.0


expenses_agent = ChatCompletionAgent(
    service=AzureChatCompletion(),
    name="ExpensesAgent",
    instructions=dedent("""
    You are an expenses agent. You can help users to manage their expenses by providing insights and recommendations.
    """),
    plugins=[ExpensesPlugin()],
)
budget_management_agent = ChatCompletionAgent(
    service=AzureChatCompletion(),
    name="BudgetManagementAgent",
    instructions=dedent("""
    You are a budget management agent. You can help users to manage their budget by providing insights and recommendations.
    """),
    plugins=[BudgetAmountPlugin(), expenses_agent],
)

# 대화형 인터페이스를 시작합니다.
while True:
    try:
        user_input = input("대화를 입력하세요 (or 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break
        print(f"👱🏻:> {user_input}")
        
        # ChatHistory를 KernelArguments에 전달하여 chat_function을 호출한다.
        response = await budget_management_agent.get_response(messages=user_input)
        print(f"🤖:> {response}")
        print("")
    except (KeyboardInterrupt, EOFError):
        break

👱🏻:> 전체 지출은 얼마야 ?


/opt/anaconda3/envs/azure-dev/lib/python3.12/site-packages/opentelemetry/sdk/_events/__init__.py:53: LogDeprecatedInitWarning: LogRecord init with `trace_id`, `span_id`, and/or `trace_flags` is deprecated since 1.35.0. Use `context` instead.
  log_record = LogRecord(
/opt/anaconda3/envs/azure-dev/lib/python3.12/site-packages/opentelemetry/sdk/_events/__init__.py:53: LogDeprecatedInitWarning: LogRecord init with `trace_id`, `span_id`, and/or `trace_flags` is deprecated since 1.35.0. Use `context` instead.
  log_record = LogRecord(


🤖:> 전체 지출 합계는 15,852,300원입니다.

필요하시면 항목별이나 날짜별, 상점별로도 더 자세히 안내해드릴 수 있습니다. 추가로 궁금한 점 있으시면 말씀해 주세요!

👱🏻:> 교보문고에서 지출한 내역은 ?


/opt/anaconda3/envs/azure-dev/lib/python3.12/site-packages/opentelemetry/sdk/_events/__init__.py:53: LogDeprecatedInitWarning: LogRecord init with `trace_id`, `span_id`, and/or `trace_flags` is deprecated since 1.35.0. Use `context` instead.
  log_record = LogRecord(


🤖:> 교보문고에서의 최근 지출 내역은 다음과 같습니다:

- 2025년 5월 23일: 246,500원 (부가세 24,650원 포함)
- 2025년 5월 31일: 163,800원 (부가세 16,380원 포함)
- 2025년 8월 21일: 164,700원 (부가세 16,470원 포함)

합계나 추가 내역이 필요하시면 말씀해 주세요!

👱🏻:> 8월 21일에 지출한 내역은 ?


/opt/anaconda3/envs/azure-dev/lib/python3.12/site-packages/opentelemetry/sdk/_events/__init__.py:53: LogDeprecatedInitWarning: LogRecord init with `trace_id`, `span_id`, and/or `trace_flags` is deprecated since 1.35.0. Use `context` instead.
  log_record = LogRecord(


🤖:> 8월 21일의 지출 내역은 다음과 같습니다:

1. 김밥천국: 198,300원 (부가세 19,830원 포함)
2. 교보문고: 164,700원 (부가세 16,470원 포함)

더 궁금한 점이나 다른 날짜의 내역이 필요하시면 말씀해 주세요!



# Image Processing - Document Intelligence

In [11]:
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.core.credentials import AzureKeyCredential
    
def ai_enrichment_analyze_receipt(path: str):
    client = DocumentIntelligenceClient(
        endpoint=os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT"),
        credential=AzureKeyCredential(os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_KEY"))
    )
    
    receipts = []
    for file in os.listdir(path):
        if not file.endswith(".png") and not file.endswith(".jpg"):
            continue

        filename = os.path.join(path, file)
        print(f"⏱️ Processing file: {filename=}...")
        with open(filename, "rb") as file:
            poller = client.begin_analyze_document("prebuilt-receipt", file)

        for receipt in poller.result().documents:
            receipts.append({
                "merchantName": receipt.fields.get("MerchantName").value_string if receipt.fields.get("MerchantName") else None,
                "transactionDate": receipt.fields.get("TransactionDate").value_date if receipt.fields.get("TransactionDate") else None,
                "transactionTime": receipt.fields.get("TransactionTime").value_time if receipt.fields.get("TransactionTime") else None,
                "tax": receipt.fields.get("TotalTax").value_currency.amount if receipt.fields.get("TotalTax") else None,
                "total": receipt.fields.get("Total").value_currency.amount if receipt.fields.get("Total") else None,
            })
        print(f"✅ Processed {filename} successfully.")
        print("------------------------------")

    return receipts

In [12]:
# receipt 이미지 파일을 분석하고, embedding 을 수행할 chunks 를 정의합니다.
_receipts = ai_enrichment_analyze_receipt("resources/receipts")

receipts = []
for receipt in _receipts:
    if receipt["total"] is None:
        print(f"Skipping receipt due to missing total or tax: {receipt}")
        continue
    receipts.append(receipt)

chunks = [
    (
        f'Receipt from {receipt["merchantName"]} on {receipt["transactionDate"]} '
        f'at {receipt["transactionTime"]}. ' if receipt["transactionTime"] else '. '
        f'Total price is {receipt["total"]} '
        f'with tax {receipt["tax"]}.' if receipt["tax"] else '. '
    ) for receipt in receipts
]

⏱️ Processing file: filename='resources/receipts/250508@청솔찬.png'...
✅ Processed resources/receipts/250508@청솔찬.png successfully.
------------------------------
⏱️ Processing file: filename='resources/receipts/250407@StayFit_법인영수증.jpg'...
✅ Processed resources/receipts/250407@StayFit_법인영수증.jpg successfully.
------------------------------
⏱️ Processing file: filename='resources/receipts/250520@창동백반.png'...
✅ Processed resources/receipts/250520@창동백반.png successfully.
------------------------------
⏱️ Processing file: filename='resources/receipts/250408@점순이하우스.png'...
✅ Processed resources/receipts/250408@점순이하우스.png successfully.
------------------------------
⏱️ Processing file: filename='resources/receipts/250513@화통소금구이.png'...
✅ Processed resources/receipts/250513@화통소금구이.png successfully.
------------------------------
⏱️ Processing file: filename='resources/receipts/250512@나주소나주곰탕.png'...
✅ Processed resources/receipts/250512@나주소나주곰탕.png successfully.
------------------------------
⏱️ P

In [13]:
from uuid import uuid4
from openai import AzureOpenAI

# receipts 에 embedding 생성합니다.
response = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
).embeddings.create(
    model=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
    input=chunks,
)
# documents 로 변환합니다.
documents = [
    {
        "rid": uuid4().hex,
        "merchantName": receipts[data.index]["merchantName"],
        "transactionDate": str((receipts[data.index]["transactionDate"]).strftime('%Y%m%d')),
        "transactionTime": int((receipts[data.index]["transactionTime"]).strftime('%H%M%S')),
        "tax": float(receipts[data.index]["tax"]) if receipts[data.index]["tax"] else None,
        "total": float(receipts[data.index]["total"]),
        "descEmbedding": data.embedding,
    } for data in response.data
]
# documents 를 업로드합니다.
search_client.upload_documents(documents)

# AI Agent with AI Search after Image Processing

In [26]:
from textwrap import dedent
from typing import Annotated
from azure.search.documents.models import VectorizedQuery
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions.kernel_function_decorator import kernel_function


class ExpensesPlugin:
    search_client = SearchClient(
        endpoint=os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
        index_name=INDEX_NAME,
        credential=AzureKeyCredential(os.getenv("AZURE_AI_SEARCH_API_KEY")),
    )
    azure_open_ai = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    )
    
    @kernel_function(name="get_total_expenses", description="Get total expenses.")
    async def get_total_expenses(self) -> float:
        results = self.search_client.search(
            search_text=None,
            select="total",
        )
        total_expenses = sum(result["total"] for result in results)
        return total_expenses

    @kernel_function(name="get_expenses", description="Get expenses for a given data.")
    async def get_expenses_by_date(
        self,
        data: Annotated[str, "Data to get expenses as date or merchant name. Use '20250821' for date or 'CU' for merchant name."],
    ) -> list[dict]:
        return self.vector_search(data)

    def vector_search(self, query: str) -> list[dict]:
        # This function performs a vector search on the Azure AI Search index.
        response = self.azure_open_ai.embeddings.create(
            model=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
            input=query,
        )
        results = self.search_client.search(
            search_text=None,
            vector_queries=[
                VectorizedQuery(
                    vector=response.data[0].embedding,
                    k_nearest_neighbors=3,
                    fields="descEmbedding",
                ),
            ],
        )
        return [result for result in results]
    
    
class BudgetAmountPlugin:
    user_budget: dict[str, float] = {}

    @kernel_function(name="set_budget_amount", description="Set a budget amount for the user.")
    async def set_user_budget_amount(
        self,
        user: Annotated[str, "The name of the user"],
        amount: Annotated[float, "The budget amount to set"],
    ) -> None:
        # This function sets the budget amount for the user.
        self.user_budget[user] = amount
        
    @kernel_function(name="get_budget_amount", description="Get the budget amount for the user.")
    async def get_user_budget_amount(
        self,
        user: Annotated[str, "The name of the user"],
    ) -> float:
        # This function retrieves the budget amount for the user.
        if user in self.user_budget:
            return self.user_budget[user]
        else:
            print(f"No budget amount set for {user}.")
            return 0.0


expenses_agent = ChatCompletionAgent(
    service=AzureChatCompletion(),
    name="ExpensesAgent",
    instructions=dedent("""
    You are an expenses agent. You can help users to manage their expenses by providing insights and recommendations.
    """),
    plugins=[ExpensesPlugin()],
)
budget_management_agent = ChatCompletionAgent(
    service=AzureChatCompletion(),
    name="BudgetManagementAgent",
    instructions=dedent("""
    You are a budget management agent. You can help users to manage their budget by providing insights and recommendations.
    """),
    plugins=[BudgetAmountPlugin(), expenses_agent],
)

# 대화형 인터페이스를 시작합니다.
while True:
    try:
        user_input = input("대화를 입력하세요 (or 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break
        print(f"👱🏻:> {user_input}")
        
        # ChatHistory를 KernelArguments에 전달하여 chat_function을 호출한다.
        response = await budget_management_agent.get_response(messages=user_input)
        print(f"🤖:> {response}")
        print("")
    except (KeyboardInterrupt, EOFError):
        break

👱🏻:> 전체 지출은 얼마야 ?


/opt/anaconda3/envs/azure-dev/lib/python3.12/site-packages/opentelemetry/sdk/_events/__init__.py:53: LogDeprecatedInitWarning: LogRecord init with `trace_id`, `span_id`, and/or `trace_flags` is deprecated since 1.35.0. Use `context` instead.
  log_record = LogRecord(


🤖:> 전체 지출은 16,479,800원입니다. 
특정 기간이나 카테고리별 지출이 궁금하시면 추가로 알려드릴 수 있습니다!

👱🏻:> 교보문고에서 지출한 내역은 ?


/opt/anaconda3/envs/azure-dev/lib/python3.12/site-packages/opentelemetry/sdk/_events/__init__.py:53: LogDeprecatedInitWarning: LogRecord init with `trace_id`, `span_id`, and/or `trace_flags` is deprecated since 1.35.0. Use `context` instead.
  log_record = LogRecord(


🤖:> 교보문고에서 지출한 내역은 다음과 같습니다:

1. 2025년 5월 23일: 246,500원 (부가세 24,650원)
2. 2025년 5월 31일: 163,800원 (부가세 16,380원)
3. 2025년 8월 21일: 164,700원 (부가세 16,470원)

총액이나 더 자세한 내역이 필요하시면 말씀해 주세요!

👱🏻:> 8월 21일에 지출한 내역은 ?


...

🤖:> 8월 21일에 지출한 내역은 다음과 같습니다:

1. 김밥천국: 198,300원 (부가세 19,830원 포함)
2. 교보문고: 164,700원 (부가세 16,470원 포함)

더 궁금하신 점이 있다면 말씀해 주세요!

👱🏻:> 창동백반에서 지출한 내역은 ?


/opt/anaconda3/envs/azure-dev/lib/python3.12/site-packages/opentelemetry/sdk/_events/__init__.py:53: LogDeprecatedInitWarning: LogRecord init with `trace_id`, `span_id`, and/or `trace_flags` is deprecated since 1.35.0. Use `context` instead.
  log_record = LogRecord(


🤖:> 창동백반에서의 지출 내역은 다음과 같습니다:

- 2025년 5월 20일: 9,000원 (부가세 818원 포함)

추가로 다른 유사한 지점의 내역도 확인 원하시면 말씀해 주세요!

👱🏻:> 5월 20일에 지출한 내역은 ?


/opt/anaconda3/envs/azure-dev/lib/python3.12/site-packages/opentelemetry/sdk/_events/__init__.py:53: LogDeprecatedInitWarning: LogRecord init with `trace_id`, `span_id`, and/or `trace_flags` is deprecated since 1.35.0. Use `context` instead.
  log_record = LogRecord(


🤖:> 2025년 5월 20일에 지출한 내역은 다음과 같습니다:

1. 창동백반: 9,000원 결제 (부가세 818원 포함)
2. ABC마트: 268,600원 결제 (부가세 26,860원 포함)

더 궁금한 점 있으시면 말씀해 주세요!

